# Simple chat-bot example

In [1]:
!pip install annoy

  Using cached annoy-1.17.0.tar.gz (646 kB)
  Running setup.py clean for annoy
Failed to build annoy
    Running setup.py install for annoy: started
    Running setup.py install for annoy: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\a.kraev\\AppData\\Local\\Temp\\pip-install-um795qdb\\annoy\\setup.py'"'"'; __file__='"'"'C:\\Users\\a.kraev\\AppData\\Local\\Temp\\pip-install-um795qdb\\annoy\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\a.kraev\AppData\Local\Temp\pip-wheel-jhwfqsvg'
       cwd: C:\Users\a.kraev\AppData\Local\Temp\pip-install-um795qdb\annoy\
  Complete output (10 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\annoy
  copying annoy\__init__.py -> build\lib.win-amd64-3.8\annoy
  running build_ext
  building 'annoy.annoylib' extension
  error: Microsoft Visual C++ 14.0 is required. 

In [3]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
#import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [13]:
#assert False

#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

<ipython-input-13-70cf6d0a615c>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 79: character maps to <undefined>

In [11]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [12]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

<ipython-input-12-4964f3fb6446>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 79: character maps to <undefined>

In [ ]:
sentences = [i for i in sentences if len(i) > 2]

In [25]:
sentences[0]

['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести']

In [17]:
modelW2V = Word2Vec(sentences=sentences, size=50, window=5, min_count=1)

In [18]:
modelFT = FastText(sentences=sentences, size=50, min_count=1, window=5)

In [19]:
w2v_index = annoy.AnnoyIndex(50 ,'angular')
ft_index = annoy.AnnoyIndex(50 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(50)
        vector_ft = np.zeros(50)
        for word in question:
            if word in modelW2V:
                vector_w2v += modelW2V[word]
                n_w2v += 1
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 100000:
            break

w2v_index.build(10)
ft_index.build(10)

True

In [20]:
w2v_index.get_nns_by_vector(np.zeros(50), 3)

[1873, 4179, 4232]

In [28]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(50)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [34]:
TEXT = "когда сменится президент"

In [35]:

get_response(TEXT, w2v_index, modelW2V, index_map)

['Скорее всего я или люди из моего окружения.Так что знакомтесь пока есть время и вожможность. \n',
 'Вам, мадам, к психиатру нужно. И как можно скорее.... \n',
 'Потому что большинство считали что это фарс чтобы подорвать репутацию новой власти Украины. Поэтому ему просто не сообщили место и день регистрации. И конечно потому что он был бы серьезным конкурентом на пост президента, его бы поддержала практически вся молодёжь. Но теперь он собирается стать мэром города Одессы, и в него есть все шансы. Дарт Вейдер выступает как интернет партия Украины, и ничего никому не обещает.. \n']

In [36]:
get_response(TEXT, ft_index, modelFT, index_map)

['Следите за новостями в Украине. Как только там начнётся - считайте, что караван верблюдов уже вышел.. . Ждите, когда он прийдёт к вам.. . А лучше - скупайте баксы.... \n',
 'Поторопился..... \n',
 'такого быть не может! если такое есть, это серьезное нарушение. тут можно и бабок срубить с ПНД. епт. \n']